In [2]:
# Importar as bibliotecas necessárias
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.naive_bayes import ComplementNB
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, f1_score, precision_score, recall_score, roc_curve
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
from io import StringIO

# Baixar recursos necessários do nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/joao/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/joao/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/joao/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Carregamento e Combinação de Múltiplos Arquivos CSV

Aqui estamos realizando o carregamento e a combinação de múltiplos arquivos CSV que estão armazenados em um repositório no GitHub. Primeiramente, definimos a URL base e geramos uma lista com os nomes dos arquivos que desejamos carregar. Em seguida, criamos URLs completas para cada arquivo CSV.

Utilizamos uma lista para armazenar os DataFrames individuais que são carregados ao fazer requisições HTTP GET para cada URL. Cada resposta é verificada para garantir que a requisição foi bem-sucedida antes de ler o conteúdo do CSV em um DataFrame. Se a requisição falhar, uma mensagem de erro é impressa.

Após carregar todos os arquivos CSV, combinamos todos os DataFrames em um único DataFrame utilizando a função `concat` do pandas. Finalmente, exibimos as primeiras linhas do DataFrame combinado para verificar se os dados foram carregados corretamente.


In [3]:
base_url = 'https://raw.githubusercontent.com/jvlopess/sentiment-analysis-naive-bayes/main/data/'
file_names = [f'tweets_part_{i}.csv' for i in range(32)]
urls = [base_url + file_name for file_name in file_names]

# Lista para armazenar os DataFrames
dataframes = []
# Baixar e carregar cada arquivo CSV em um DataFrame
for url in urls:
    response = requests.get(url)
    if response.status_code == 200:
        df = pd.read_csv(StringIO(response.text))
        dataframes.append(df)
    else:
        print(f"Failed to fetch {url}")

# Combinar todos os DataFrames em um único DataFrame
df = pd.concat(dataframes, ignore_index=True)

# Exibir as primeiras linhas do DataFrame combinado
df.head()


,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


### Adição de Títulos de Colunas ao DataFrame

Neste bloco de código, adicionamos títulos de colunas ao DataFrame `df`, uma vez que os dados carregados não possuem cabeçalhos definidos.

Primeiro, definimos os títulos das colunas de acordo com a estrutura dos dados:
- **label:** Indica a classificação do tweet (sentimento).
- **time:** Hora em que o tweet foi postado.
- **date:** Data em que o tweet foi postado.
- **query:** Informação de consulta (geralmente vazia ou irrelevante).
- **username:** Nome de usuário do autor do tweet.
- **tweet:** O texto do tweet.

Após definir os títulos das colunas, aplicamos essas definições ao DataFrame `df` e, em seguida, exibimos as primeiras linhas do DataFrame para verificar se os títulos foram aplicados corretamente.


In [4]:
 # As the data has no column titles, we will add our own
df.columns = ["label", "time", "date", "query", "username", "tweet"]

df.head()

,label,time,date,query,username,tweet
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


### Embaralhamento e Redução do Conjunto de Dados

Neste bloco de código, estamos embaralhando e reduzindo o tamanho do conjunto de dados para facilitar o processamento e a análise.

1. **Embaralhamento do DataFrame:**
   - Utilizamos `df.sample(frac=1)` para embaralhar as linhas do DataFrame `df` aleatoriamente. O parâmetro `frac=1` indica que todas as linhas devem ser incluídas no embaralhamento.

2. **Redução do Tamanho do Conjunto de Dados:**
   - Reduzimos o DataFrame para incluir apenas as primeiras 200.000 linhas utilizando `df = df[:200000]`. Isso é útil para trabalhar com um subconjunto gerenciável dos dados, especialmente em cenários onde o conjunto de dados original é muito grande.

3. **Verificação da Estrutura do Conjunto de Dados:**
   - Imprimimos a forma do DataFrame reduzido com `print("Dataset shape:", df.shape)`, o que nos permite verificar o número de linhas e colunas após o embaralhamento e a redução.

Essas etapas ajudam a garantir que o conjunto de dados seja aleatório e de um tamanho adequado para análises subsequentes.


In [5]:
df = df.sample(frac=1)
df = df[:200000]
print("Dataset shape:", df.shape)

Dataset shape: (200000, 6)


### Verificação dos Valores Únicos na Coluna de Rótulos

Neste bloco de código, estamos verificando os valores únicos presentes na coluna `label` do DataFrame `df`.

- **Objetivo:**
  - O objetivo é identificar as diferentes classes ou rótulos presentes no conjunto de dados, que indicam a classificação do sentimento dos tweets.

- **Método:**
  - Utilizamos `df['label'].unique()` para extrair uma lista de valores únicos da coluna `label`.

- **Importância:**
  - Esta etapa é crucial para entender a distribuição dos dados e garantir que os rótulos de classificação sejam consistentes com as expectativas (por exemplo, 0 para sentimentos negativos e 4 para sentimentos positivos).


In [6]:
df['label'].unique()

array([0, 4])

### Currently (0=negative,4=Positive) changing the notation to (0=Negative,1=Positive)

In [7]:
df['label']=df['label'].replace(4,1)
df

,label,time,date,query,username,tweet
533348,0,2197107207,Tue Jun 16 13:21:06 PDT 2009,NO_QUERY,EddieHdz,In North Hollywood surviving the flaming sun!!!
209109,0,1973889928,Sat May 30 12:01:58 PDT 2009,NO_QUERY,PhoebeKaye11,ouch
445216,0,2068058373,Sun Jun 07 13:25:36 PDT 2009,NO_QUERY,SarFRENCH,this is insane
1281673,1,2001760416,Tue Jun 02 01:59:05 PDT 2009,NO_QUERY,Saaaamie,another deal another day
492633,0,2184312398,Mon Jun 15 15:43:34 PDT 2009,NO_QUERY,dubya_b,I no longer have an apartment.
...,...,...,...,...,...,...
688840,0,2251620074,Sat Jun 20 04:04:37 PDT 2009,NO_QUERY,franhaw,was not able to satisfy her hot pot craving to...
799812,0,2329141278,Thu Jun 25 10:23:56 PDT 2009,NO_QUERY,Gigiborja,@juliejolie Maw is in brussels again
148593,0,1883179071,Fri May 22 08:08:07 PDT 2009,NO_QUERY,thestolenfork,@TheRealRiquee We need to get together one day...
170153,0,1962836388,Fri May 29 11:45:50 PDT 2009,NO_QUERY,robertsm85,I have so much stuff in my car that I can just...


### Remoção de Colunas Desnecessárias

Neste bloco de código, estamos simplificando o DataFrame `df` removendo colunas que não são relevantes para a análise de sentimentos dos tweets.

1. **Remoção de Colunas `date`, `query` e `username`:**
   - Utilizamos `df.drop(['date', 'query', 'username'], axis=1, inplace=True)` para remover as colunas `date`, `query` e `username`. Estas colunas não são necessárias para a tarefa de classificação de sentimentos e são removidas para simplificar o DataFrame.

2. **Remoção da Coluna `time`:**
   - Removemos a coluna `time` separadamente utilizando `df.drop('time', axis=1, inplace=True)`, pois ela também não é relevante para a análise de sentimentos.

3. **Exibição das Primeiras Linhas do DataFrame Modificado:**
   - Utilizamos `df.head(10)` para exibir as primeiras 10 linhas do DataFrame após a remoção das colunas. Isso nos permite verificar que as colunas foram removidas corretamente e que o DataFrame agora contém apenas as colunas `label` e `tweet`.

A remoção dessas colunas ajuda a focar apenas nos dados necessários para a análise, facilitando o processamento e a modelagem subsequentes.


In [8]:
df.drop(['date','query','username'], axis=1, inplace=True)
df.drop('time', axis=1, inplace=True)
df.head(10)

,label,tweet
533348,0,In North Hollywood surviving the flaming sun!!!
209109,0,ouch
445216,0,this is insane
1281673,1,another deal another day
492633,0,I no longer have an apartment.
1465527,1,@iadiedee you're welcome
373198,0,Have to clean my house before the estate agent...
863751,1,Pilates and yoga build strong core muscles to ...
64958,0,"@modernsinglemom no, unfortunately they have t..."
757999,0,@jawilson thanks but I don't ever want to live...


### Função para Limpeza dos Tweets

Neste bloco de código, definimos a função `process_tweets` para realizar o pré-processamento dos textos dos tweets. Esta função aplica várias etapas de limpeza e transformação para preparar os dados para a análise de sentimentos.

1. **Conversão para Minúsculas:**
   - `tweet = tweet.lower()`
   - Converte todo o texto do tweet para letras minúsculas, garantindo consistência e evitando duplicidades baseadas em diferenças de capitalização.

2. **Remoção de URLs:**
   - `tweet = re.sub(r'http\S+|www.\S+', '', tweet)`
   - Remove quaisquer URLs presentes no tweet, pois elas geralmente não contribuem para a análise de sentimentos.

3. **Remoção de Menções:**
   - `tweet = re.sub(r'@\w+', '', tweet)`
   - Remove menções a outros usuários do Twitter, que começam com '@', para focar no conteúdo textual principal.

4. **Remoção de Números:**
   - `tweet = re.sub(r'\d+', '', tweet)`
   - Remove todos os números do tweet, pois geralmente não são úteis para a análise de sentimentos.

5. **Remoção de Pontuações:**
   - `tweet = tweet.translate(str.maketrans('', '', string.punctuation))`
   - Remove todos os sinais de pontuação para simplificar o texto.

6. **Tokenização:**
   - `tokens = word_tokenize(tweet)`
   - Divide o texto em tokens individuais (palavras), facilitando o processamento subsequente.

7. **Remoção de Stop Words:**
   - `tokens = [word for word in tokens if word not in stopwords.words('english')]`
   - Remove stop words, que são palavras comuns (como "and", "the", etc.) que não adicionam significado relevante ao texto.

8. **Lematização:**
   - `lemmatizer = WordNetLemmatizer()`
   - `tokens = [lemmatizer.lemmatize(word) for word in tokens]`
   - Reduz as palavras às suas formas base ou raiz (por exemplo, "running" para "run"), ajudando a normalizar o texto.

9. **Recomposição do Texto:**
   - `return ' '.join(tokens)`
   - Reúne os tokens processados em uma string única, representando o texto do tweet limpo e preparado para análise.

Essa função de limpeza garante que os textos dos tweets estejam em um formato consistente e livre de ruídos, facilitando a análise e melhorando o desempenho dos modelos de aprendizado de máquina.


In [9]:
def process_tweets(tweet):
    # Lower case
    tweet = tweet.lower()
    # Remove URLs
    tweet = re.sub(r'http\S+|www.\S+', '', tweet)
    # Remove mentions
    tweet = re.sub(r'@\w+', '', tweet)
    # Remove números
    tweet = re.sub(r'\d+', '', tweet)
    # Remove pontuações
    tweet = tweet.translate(str.maketrans('', '', string.punctuation))
    # Tokenização
    tokens = word_tokenize(tweet)
    # Remover stop words
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    # Lematização
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

### Aplicação da Limpeza dos Tweets e Preparação dos Dados

Neste bloco de código, aplicamos a função de limpeza aos tweets, realizamos a vetorização dos textos e preparamos os dados para treinamento e teste.

1. **Aplicação da Função de Limpeza:**
   - `df['tweet'] = df['tweet'].apply(process_tweets)`
   - Aplica a função `process_tweets` a cada tweet na coluna `tweet` do DataFrame `df`. Isso transforma os tweets brutos em textos limpos e padronizados, prontos para a vetorização.

2. **Vetorização usando CountVectorizer:**
   - `vectorizer = CountVectorizer(max_features=5000)`
   - `X = vectorizer.fit_transform(df['tweet'])`
   - Utiliza `CountVectorizer` para converter os textos dos tweets em uma matriz de contagens de palavras. Limitamos o número de características (palavras) a 5000 para manter a complexidade gerenciável.
   - `X` é a matriz de características resultante da vetorização.

3. **Preparação das Variáveis Dependente e Independente:**
   - `y = df['label']`
   - Define `y` como a variável dependente contendo os rótulos (sentimentos) dos tweets.

4. **Divisão do Conjunto de Dados:**
   - `X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)`
   - Divide os dados em conjuntos de treinamento e teste. Utilizamos 80% dos dados para treinamento (`X_train`, `y_train`) e 20% para teste (`X_test`, `y_test`).
   - O parâmetro `random_state=42` garante que a divisão dos dados seja reprodutível.

Essas etapas preparam os dados textuais para serem utilizados em modelos de aprendizado de máquina, convertendo-os de texto cru para uma representação numérica que pode ser processada por algoritmos.


In [11]:
# Aplicar a limpeza dos tweets
df['tweet'] = df['tweet'].apply(process_tweets)

# Vetorização usando CountVectorizer
vectorizer = CountVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['tweet'])
y = df['label']

# Dividir o conjunto de dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)